<h1 align="center"> Generative AI Hackathon</h1>
<table align="center">
    <td>
        <a href="https://colab.research.google.com/github/teamdatatonic/gen-ai-hackathon/blob/main/hackathon.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo">
            <span style="vertical-align: middle;">Run in Colab</span>
        </a>
    </td>
    <td>
        <a href="https://github.com/teamdatatonic/gen-ai-hackathon/blob/main/hackathon.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
            <span style="vertical-align: middle;">View on GitHub</span>
        </a>
    </td>
    <td>
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/teamdatatonic/gen-ai-hackathon/main/hackathon.ipynb">
            <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 
            <span style="vertical-align: middle;">Open in Vertex AI Workbench</span>
        </a>
    </td>
</table>
<hr>

 **🔵 Your main goal with this notebook:** Learn about Generative AI by building your own Knowledge Worker using Python and LangChain!



# Introduction

This notebook walks you through the challenge of implementing a **Knowledge Worker** for your organisation using **Generative AI**!

**Why a knowledge worker?** Decentralized data across internal and external databases results in time wasted as workforce tries to find required information and transform into insights. A knowledge worker can consolidate this information, then answer queries in natural language providing summarisation and sources.

![Q&A Chain intro](https://github.com/teamdatatonic/gen-ai-hackathon/blob/bca8120f1408be1895309517a7a4d693035b940b/assets/qa_intro.png?raw=true)

🔵 **Definition of your main goal:** Implement a knowledge worker to enable users in your company to perform Q&A, in natural language, upon a knowledge base.
In this way, you'll centralise company data for easy access in a user-friendly manner, boosting productivity.
As such, you'll create a knowledge worker fine-tuned to your data domain.
This app will only have access to specific knowledge such as public data about your company available on your company's website and unstructured documents (websites, PDF, Word, text ...).

While solving the tasks as instructed in this notebook, you'll familiarise yourself with common concepts and tools for Generative AI including:

* The Open-Source tool LangChain
* Large Language Models (LLMs)
* Vertex AI Search (a.k.a Enterprise Search)
* Prompts and Prompt Engineering
* Text Embeddings and Vector Databases


Ultimately, this notebook details how to get started with LangChain and Vertex AI to do Retrieval Augmented Generation (RAG), and walks through setting up a knowledge worker on Google Cloud and Vertex AI.

## Implementing a knowledge worker

When creating a knowledge worker, you'll recall that Large Language Models (LLMs) can be tuned for a variety of tasks such as text summarization, answering questions, and generating new content (and many more!).
When it comes to tuning approaches, you'll have the choice between:

**A) Zero-shot learning:** Use LLMs directly without providing additional data or fine-tuning the model.

**B) Few-shot learning:** Provide a select number of input examples when using LLM to improve the quality of outputs.

**C) Model Fine-tuning:** Fine-tune certain (or additional) layers in the LLM by training the model on provided training data.

Instead of training LLMs using your own data (ie. fine-tuning), it is far easier and more effective to adapt the LLM to your use-case by prompt engineering only (ie. tuning).
Thus, methods A) and B) are more applicable for creating your first knowledge worker.

A knowledge worker can be approached in two stages:

1. Embedding knowledge from diverse sources.
    * Load our dataset.
    * Shard our documents (e.g.: by paragraph, per 1000 tokens, etc.)
    * Embed the documents in Vertex AI Search or a vector store.
2. Querying a LLM which is aware of your relevant knowledge to answer questions.
    * Locating relevant documents from Vertex AI Search or a vector store.
    * Asking the LLM our query, providing relevant knowledge as context to generate an answer.

![Q&A Chain Flow](https://github.com/teamdatatonic/gen-ai-hackathon/blob/bca8120f1408be1895309517a7a4d693035b940b/assets/qa_flow.jpeg?raw=true)
First, documents (websites, Word documents, databases, Powerpoints, PDFs, etc.) are loaded and split into chunks. Fragmenting is important for three reasons:

1. There are technical restrictions on how much data (tokens) can be fed into an LLM at once, meaning the context + system prompt + chat history + user prompt must fit within the token limit.
2. Most LLM APIs operate on a per-token pricing model, meaning it is cost-effective to limit the size / amount of data provided to the LLM per query.
3. Contextual information should be relevant to the user query, meaning it is optimal to provide only relevant snippets from documents, making the answer more relevant whilst saving costs as per (1) and (2).

Next, these document shards are embedded within a vector store. Embedding a document means to align it within a multi-dimension space, which can then be searched according to user queries to find relevant documents. Document relevancy scoring can be as simple as a K-neighbours search, since embedded documents with similarity (as percieved by the LLM embedding model) will be proximate within the search space.

![A typical ingestion chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/7f37d477b18ace5912d34b0574512559d7a457ed/assets/typical-ingestion-chain.png?raw=true)

Once the vector store is created, a user can query the knowledge base using natural language questions. Relevant documents related to the query are found in the vector store by embedding the user query and finding local documents. These snippets of text are provided to the LLM (alongside the user query, chat history, prompt engineering, etc.) which parses the information to generate an answer.

![A typical query chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/7f37d477b18ace5912d34b0574512559d7a457ed/assets/typical-query-chain.png?raw=true)

## Running this workshop
* Execute each code snippet sequentially. This lab is designed so certain steps (like defining functions or importing modules) are performed once, so each new code cell builds upon previous cells.
* If the notebook crashes or you want to restart the notebook later, make sure you execute all cells prior to where you left off.
* Executing cells out of order can lead to errors, so your first step when debugging should be to ensure all previous code cells have been run.
* This workshop can be completed independently, but Datatonic workshop leaders are available to discuss tasks, debug issues, or have a chat about generative AI!

## Prerequisites
### Install Python dependencies
We've developed a python module specifically for this workshop. Installing this one package also installs other dependencies, such as LangChain (a LLM framework) and GradIO (a web UI framework).

In [1]:
%pip install --quiet "git+https://github.com/teamdatatonic/gen-ai-hackathon.git@feat/add-vertex-ai-search#subdirectory=dt_gen_ai_hackathon_helper"


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**❗ Restart the Python kernel:** Ensure that your environment can access the newly installed dependencies. Continue after the restart from the `Setup cloud project` step. 

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**❗ Note:** If your kernel doesn't restart automatically, click the "Restart Runtime" button above your notebook.
If you dont see a restart button, go to the "Runtime" toolbar tab then "Restart Runtime". After restarting, continue executing the project from below this cell.

## Accessing the Vertex AI Endpoint

Currently, Vertex AI LLMs are accessible via Google Cloud projects. We will access the Vertex AI endpoint via a service account.

1. Upload the Google Application Credentials `.json` file sent to your email to the notebook filesystem.
2. Set the variable `GOOGLE_APPLICATION_CREDENTIALS` with the filepath (**❗ Note:** the `/content/` folder is where uploaded files are stored by default).

In [1]:
import os

# @title Set project credentials. { run: "auto", display-mode: "form" }
# @markdown Set the filepath to the `.json` credentials file.
GOOGLE_APPLICATION_CREDENTIALS = "secrets/credentials.json"  # @param {type:"string"}
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

!gcloud auth activate-service-account --key-file={GOOGLE_APPLICATION_CREDENTIALS}

Activated service account credentials for: [dt-gen-ai-hackathon-sa@dt-gen-ai-hackathon-dev.iam.gserviceaccount.com]


In [2]:
# @markdown Set the Google Cloud project ID.
from google.cloud import aiplatform

PROJECT_ID = "dt-gen-ai-hackathon-dev"  # @param {type:"string"}
aiplatform.init(project=PROJECT_ID)

!gcloud config set project {PROJECT_ID}

Updated property [core/project].


# Section 1: Implementing a knowledge worker with Vertex AI Search

Creating a custom knowledge worker is similar to your first step when learning a new programming language.
As such your first challenge is to create a “Hello World” program, however, adapted to LLMs which is way more exciting!

With a few lines of code, you'll:
* Load documents with information about a/your company
* Store documents in Vertex AI Search
* Use an LLM to answer queries about Alphabet's investors reports

**❗ All of these steps can be achieved in a few lines of Python, or can be equally achieved through the UI.**

#### **➡️ Your task:**
Do the following steps in the Vertex AI Search UI to create a data store and an application.

## 1.1 Setting up Vertex AI Search

Vertex AI Search and Conversation is a comprehensive platform aimed at simplifying the development of search and conversational applications for enterprises. Key features include:

**Rapid Development:** The platform's out-of-the-box capabilities eliminate the need for tasks like data chunking and generating embeddings, allowing developers to create apps within hours or days, even without prior machine learning experience.

**Flexible Integration:** Whether you're a beginner or an advanced developer, the platform caters to varying needs. It offers direct API integration and allows for deep customizations via specialized offerings like Vector Search or Enterprise Knowledge Graph.

**Personalized Interactions:** Applications developed using Vertex AI support multi-modal (text, images, voice) and multi-turn conversations. This enables more personalized and efficient user experiences, supported by Google’s foundation models.

**Actionable Insights:** Vertex AI extensions enable chatbots to access and interact with both internal and third-party applications, allowing for real-time actions like booking a calendar or purchasing a flight.

**Data Control and Compliance:** Enterprises have full control over their data and user interactions, which are stored in their own cloud instance. The platform also adheres to industry compliance standards like HIPAA, DRZ, and CMEK.



![vertex_ai_search](https://github.com/teamdatatonic/gen-ai-hackathon/blob/9ef675fa333cd2de48a084780940e2cc5e08c0ed/assets/vertex_ai_search.png?raw=true)


### Create a Vertex AI Search data store with documents
Create a Vertex AI Search data store to host unstructured data (pdf documents)

1 - On the Search & Conversation page, click `NEW DATA STORE`.


![Select NEW DATA STORE](https://github.com/teamdatatonic/gen-ai-hackathon/blob/cc33b4034411d494815e65797f21ebc51a28e046/assets/new_data_store.png?raw=true)

2 - Select a data source of type Cloud Storage. 

3 - Add the folder path `dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021`and select the data kind `Unstructured documents`. gs:// is not required.

![Select kind of data](https://github.com/teamdatatonic/gen-ai-hackathon/blob/b0d89b56d4e6063d7a7de8c590ca6e886c9a0883/assets/select_kind_of_data.png?raw=true)

4 - Location should be `global (Global)` and the data store name `dt-gen-ai-hackathon-<TEAM NUMBER>`.e.g. `dt-gen-ai-hackathon-t1`.


### Create a Vertex AI Search application
Create a Vertex AI Search application to host your knowledge worker.

1 - On the Search & Conversation page, click `NEW APP`.

2 - **Type**: Select app type Search.

3 - **Configuration**: Maintain the default values with `gobal (Global)` and both features activated. Add the app name `app_<TEAM NUMBER>`.e.g. `app_t1` and click `CONTINUE`.

![App configuration](https://github.com/teamdatatonic/gen-ai-hackathon/blob/fc930a5159026ff7ee7ecd8d9440cd1bde0f4a22/assets/app_configuration.png?raw=true)

4 - **Data**: Select the data store created in the previous step and click `CREATE`.

🎉 Congratulations! 🎉 You've created a Vertex AI Search data store and application. Now, you can use the Vertex AI Search UI to query your data store and see the results.

## 1.2 LangChain retrieval Q&A chains

LangChain is a Python framework for developing applications using language models.
It abstracts the connection between applications and LLMs, allowing a loose coupling between code and specific providers like Google PaLM.

We will demonstrate the use of three types of LangChain retrieval Q&A chains:

- RetrievalQA
- RetrievalQAWithSourcesChain
- ConversationalRetrievalChain

First, we initialize a Vertex AI Language Model (LLM) and a LangChain `retriever` to fetch documents from our Vertex AI Search engine.

In the case of Q&A chains, our retriever is directly passed to the chain, enabling it to function automatically without requiring any additional configuration.

Behind the scenes, the search query is initially passed to the retriever. The retriever performs a search and returns relevant document snippets. These snippets are then used as context for the prompt executed by the LLM.

In [3]:
from langchain.llms import VertexAI
from langchain.retrievers import GoogleCloudEnterpriseSearchRetriever

# Get this value from the Vertex AI Search UI
# Here add your data store id
data_store_id = "dt-gen-ai-hackathon-t1_1696592331863"
MODEL = "text-bison@001"

llm = VertexAI(model_name=MODEL, temperature=0.0)

retriever = GoogleCloudEnterpriseSearchRetriever(
    project_id=PROJECT_ID, search_engine_id=data_store_id
)

### RetrievalQA Chain
This is the simplest document Q&A chain offered by LangChain.

Several different chain types are available, as listed [here](https://docs.langchain.com/docs/components/chains/index_related_chains).

In these examples, we use the 'stuff' type, which simply inserts all the document snippets into the prompt. This approach has the advantage of requiring only a single LLM call, making it faster and more cost-efficient.

However, this method comes with a drawback: if we have a large number of search results, we run the risk of exceeding the token limit for our prompt or truncating useful information.

Other chain types, such as 'map_reduce' and 'refine,' employ an iterative process. These types make multiple LLM calls, taking individual document snippets one at a time and refining the answer iteratively.

In [4]:
from langchain.chains import RetrievalQA

search_query = "Give me the name of the CEO of DeepMind?"

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)
retrieval_qa.run(search_query)

'The CEO of DeepMind is Demis Hassabis.'

If we set `return_source_documents=True` as an optional parameter when constructing the chain, we can examine the document snippets returned by the retriever. This feature is particularly useful for debugging, as the relevance of these snippets to the answer may not always be immediately obvious.

In [5]:
from dt_gen_ai_hackathon_helper.formatter_helper import formatter_helper

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

results = retrieval_qa({"query": search_query})
formatter_helper.format_results(results)

*******************************************************************************
Answer: The CEO of DeepMind is Demis Hassabis.
Used 5 relevant documents.
*******************************************************************************
-------------------------------------------------------------------------------
Document 1
-------------------------------------------------------------------------------
Source of content: gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021/2015_google_annual_report.pdf
-------------------------------------------------------------------------------
to implement segment reporting for our Q4 results,
where Google financials will be provided separately
than those for the rest of Alphabet businesses as
a whole.
This new structure will allow us to keep tremendous
focus on the extraordinary opportunities we have
inside of Google. A key part of this is Sundar Pichai.
Sundar has been saying the things I would have
said (and sometimes better!) f

#### **➡️ Your task:**
Find another and more direct way to access the source documents returned by the retriever.

In [ ]:
# Your code here

In [6]:
# ANSWER
from langchain.chains import RetrievalQAWithSourcesChain

retrieval_qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

retrieval_qa_with_sources({"question": search_query}, return_only_outputs=True)

{'answer': 'The CEO of DeepMind is Demis Hassabis.\n',
 'sources': '2016_google_annual_report.pdf'}

This variant delivers both the answer to the question and the source documents used for generating that answer, doing so in a simpler manner than using `return_source_documents=True`.

### ConversationalRetrievalChain
The ConversationalRetrievalChain remembers and uses previous questions to enable a chat-like discovery process. To utilize this chain, we need to provide a memory class that stores and passes the previous messages to the LLM as context. For this purpose, we use the ConversationBufferMemory class that comes with Langchain.

In [7]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversational_retrieval = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=retriever, memory=memory
)

search_query = "What were alphabet revenues in 2021?"

result = conversational_retrieval({"question": search_query})
print(result["answer"])

Alphabet's revenues in 2021 were $257,637 million.


In [8]:
new_query = "What about costs and expenses?"
result = conversational_retrieval({"question": new_query})
print(result["answer"])

Alphabet's costs and expenses in 2021 were $13,200 million.


In [9]:
new_query = "Is this more than in 2020?"

result = conversational_retrieval({"question": new_query})
print(result["answer"])

Yes, Alphabet's revenues in 2021 were more than in 2020. In 2021, Alphabet's revenues were $257,637 million, while in 2020, Alphabet's revenues were $182,527 million. This represents a 39% increase in revenues from 2020 to 2021.


## 1.3 Prompt engineering

![Q&A Chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/bca8120f1408be1895309517a7a4d693035b940b/assets/stuff-chain.jpeg?raw=true)

As outlined before, the creation of prompts is essential to adapt LLMs for your given use case.
**Prompt engineering** is a method of zero-shot fine-tuning for large language models.
By prompting a LLM with contextual information about its purpose, the model can simulate a variety of situations, such as a customer assistant chatbot, a document summariser, a translator, etc.

In this use case, we prompt our model to respond as a conversational Q&A chatbot.
Prompt engineering can be especially useful for introducing guard rails to an application - in this template we tell the model to not respond to queries it lacks the information to answer, as users will trust the application to provide factual replies, so rejecting a query is preferable to outputting false information.

You can use the prompt and code cells below for your knowledge worker.

#### **➡️ Your task:**
Execute and study the code cell below. Pay attention to the prompt being defined. 
What elements do you notice in the prompt?
How is the prompt used in the chain? Once you have a good understanding of the code, try defining your own prompt.

In [10]:
SECTION_01_BASIC_TEMPLATE = """\
You are a helpful chatbot designed to perform Q&A on a set of documents.
Always respond to users with friendly and helpful messages.
Your goal is to answer user questions using relevant sources.

You were developed by Datatonic, and are powered by Google's PaLM-2 model.

In addition to your implicit model world knowledge, you have access to the following data sources:
- Company documentation.

If a user query is too vague, ask for more information.
If insufficient information exists to answer a query, respond with "I don't know".
NEVER make up information.
"""


In [ ]:
# Your code here

Once we have connected our Vertex AI Search data store and prompt, we can define our LangChain.

In all of the previous examples we used the default prompt that comes with langchain.

We can inspect our chain object to discover the wording of the prompt template being used.

We may find that this is not suitable for our purposes, and we may wish to customise the prompt, for example to present our results in a different format, or to specify additional constraints.

In [11]:
# Check LangChain's default prompt
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

print(qa.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


#### **➡️ Your task:**
 Create a yes or no question prompt. Let's modify the prompt to return an answer in a single word (useful for yes/no questions). We will constrain the LLM to say 'I don't know' if it cannot answer.

We want to create a new prompt_template and pass it in using the template argument. Extra points for creating the prompt as a two-string template, maintaining the BASIC_TEMPLATE and adding a second part to it.

We want the answer to provide two things:
```
short_answer: Just one word
detais: One sentence with short explanation
```

In [12]:
# Your code here

In [13]:
# ANSWER
SECTION_01_A_TEMPLATE = """\
The answer should consist of:
Short answer: 1 word. Just a yes or no answer. Prefixed with Short answer:.
Details: 1 sentence. A short explanation of the answer. Prefixed with Details:.

Context: {context}

Question: {question}
Helpful Answer:
"""

In [14]:
from langchain.prompts import PromptTemplate

prompt_a = PromptTemplate(
    template=SECTION_01_BASIC_TEMPLATE + "\n" + SECTION_01_A_TEMPLATE
    , input_variables=["context", "question"]
)
qa_chain = RetrievalQA.from_llm(
    llm=llm, prompt=prompt_a, retriever=retriever, return_source_documents=True
)

In [15]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

You are a helpful chatbot designed to perform Q&A on a set of documents.
Always respond to users with friendly and helpful messages.
Your goal is to answer user questions using relevant sources.

You were developed by Datatonic, and are powered by Google's PaLM-2 model.

In addition to your implicit model world knowledge, you have access to the following data sources:
- Company documentation.

If a user query is too vague, ask for more information.
If insufficient information exists to answer a query, respond with "I don't know".
NEVER make up information.

The answer should consist of:
Short answer: 1 word. Just a yes or no answer. Prefixed with Short answer:.
Details: 1 sentence. A short explanation of the answer. Prefixed with Details:.

Context: {context}

Question: {question}
Helpful Answer:


In [16]:
search_query = "Were 2020 EMEA revenues higher than 2020 APAC revenues?"

results = qa_chain({"query": search_query})
formatter_helper.format_results(results)

*******************************************************************************
Answer: Short answer: Yes.
Details: EMEA revenues were $55,370 million in 2020, while APAC revenues were $32,550 million.
Used 3 relevant documents.
*******************************************************************************
-------------------------------------------------------------------------------
Document 1
-------------------------------------------------------------------------------
Source of content: gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021/20201030_alphabet_10Q.pdf
-------------------------------------------------------------------------------
The following table presents the foreign exchange effect on our international revenues and total revenues (in
millions, except percentages, unaudited):

Three Months Ended

Nine Months Ended

September 30,

September 30,

2019

2020

2019

2020

EMEA revenues

$ 12,565 $ 13,924 $ 36,546 $ 38,132

Exclude foreign exchange e

## 1.4 Ingesting documents to Vertex AI Search

Now, we are going to test querying our knowledge worker with a new set of documents. First, we will perform a query without the new documents to see what the results look like. Then, we will ingest the new documents and perform the same query again to see how the results change with the ingested data related to the query, which is from the year 2022.

In [17]:
search_query = "Were 2022 EMEA revenues higher than 2022 APAC revenues?"

results = qa_chain({"query": search_query})
formatter_helper.format_results(results)

*******************************************************************************
Answer: Short answer: Yes
Details:
EMEA revenues in 2022 were $82,062 million, while APAC revenues were $47,024 million.
Used 5 relevant documents.
*******************************************************************************
-------------------------------------------------------------------------------
Document 1
-------------------------------------------------------------------------------
Source of content: gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2022_2023/20220427_alphabet_10Q.pdf
-------------------------------------------------------------------------------
Constant currency revenue percentage change is calculated by determining the change in current period
revenues over prior year comparable period revenues where current period foreign currency revenues are
translated using prior year comparable period exchange rates and hedging effects are excluded from revenues of
both peri

In [18]:
import logging
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine

logging.basicConfig(level=logging.INFO)


def ingest_documents(
        project_id: str,
        location: str,
        data_store_id: str,
        gcs_uri: Optional[str] = None,
        bigquery_dataset: Optional[str] = None,
        bigquery_table: Optional[str] = None,
        data_schema: str = "content",
) -> str:
    """
    Ingest documents into a data store.
    Args:
        project_id: The project ID of the project to import documents into.
        location: The location of the data store to import documents into.
        data_store_id: The ID of the data store to import documents into.
        gcs_uri: The GCS URI of the documents to import.
        bigquery_dataset: The BigQuery dataset of the documents to import.
        bigquery_table: The BigQuery table of the documents to import.
        data_schema: The data schema of the documents to import.
    Returns:
        The operation name of the import documents operation.
    """
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.DocumentServiceClient(client_options=client_options)

    # The full resource name of the search engine branch.
    # e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}/branches/{branch}
    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",
    )

    print(parent)

    if gcs_uri:
        request = discoveryengine.ImportDocumentsRequest(
            parent=parent,
            gcs_source=discoveryengine.GcsSource(
                input_uris=[gcs_uri], data_schema=data_schema
            ),
            # Options: `FULL`, `INCREMENTAL`
            reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
        )
    else:
        request = discoveryengine.ImportDocumentsRequest(
            parent=parent,
            bigquery_source=discoveryengine.BigQuerySource(
                project_id=project_id,
                dataset_id=bigquery_dataset,
                table_id=bigquery_table,
                data_schema="custom",
            ),
            # Options: `FULL`, `INCREMENTAL`
            reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
        )

    # Make the request
    operation = client.import_documents(request=request)

    print(f"Waiting for operation to complete: {operation.operation.name}")
    response = operation.result()

    # Once the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

    # Handle the response
    print(response)
    print(metadata)

    return operation.operation.name

Run the cell below and then go to the Vertex Search UI to see the original data store. Check in `Activity` to see how new data has been added to the store.

Then, use the cells below to query this new data.

In [19]:
# from dt_gen_ai_hackathon_helper.vertex_ai_search import vertex_ai_search
location = "global"
gcs_uri = "gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2022_2023/*.pdf"

ingest_documents(project_id=PROJECT_ID, location=location, data_store_id=data_store_id, gcs_uri=gcs_uri,
                 data_schema="content")

projects/dt-gen-ai-hackathon-dev/locations/global/dataStores/dt-gen-ai-hackathon-t1_1696592331863/branches/default_branch
Waiting for operation to complete: projects/592911206780/locations/global/collections/default_collection/dataStores/dt-gen-ai-hackathon-t1_1696592331863/branches/0/operations/import-documents-331549544055625130
error_config {
  gcs_prefix: "gs://592911206780_eu_import_content/errors331549544055627627"
}

create_time {
  seconds: 1696860789
  nanos: 546778000
}
update_time {
  seconds: 1696860790
  nanos: 828998000
}
success_count: 15


'projects/592911206780/locations/global/collections/default_collection/dataStores/dt-gen-ai-hackathon-t1_1696592331863/branches/0/operations/import-documents-331549544055625130'

In [20]:
search_query = "Were 2022 EMEA revenues higher than 2022 APAC revenues?"

results = qa_chain({"query": search_query})
formatter_helper.format_results(results)

*******************************************************************************
Answer: Short answer: Yes
Details:
EMEA revenues in 2022 were $82,062 million, while APAC revenues were $47,024 million.
Used 5 relevant documents.
*******************************************************************************
-------------------------------------------------------------------------------
Document 1
-------------------------------------------------------------------------------
Source of content: gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2022_2023/20220427_alphabet_10Q.pdf
-------------------------------------------------------------------------------
Constant currency revenue percentage change is calculated by determining the change in current period
revenues over prior year comparable period revenues where current period foreign currency revenues are
translated using prior year comparable period exchange rates and hedging effects are excluded from revenues of
both peri

## 1.5 Building the user interface

As building a UI is outside of the scope for this hackathon, a templated GUI using [Gradio](https://gradio.app/) is provided for your knowledge worker.

#### **➡️ Your task:**
Execute the cells below to launch the user interface.

**❗ Note:** The following cell will run until manually stopped. Remember to halt it before moving onto the next task.

In [41]:
def create_qa_vertex_ai_search_chain(k=10, temperature=0.0):
    """Create a Q&A conversation chain using the VertexAI LLM.

    Arguments:
        vector_store (object): The vectorstore containing our knowledge.
        condense_question_prompt (PromptTemplate): The prompt template used to prompt engineer our LLM to respond in a certain tone, etc.
        k (int): the 'k' value indicates the number of sources to use per query. 'k' as in 'k-nearest-neighbours' to the query in the embedding space.
        temperature (float): the degree of randomness introduced into the LLM response.
    """
    retriever = GoogleCloudEnterpriseSearchRetriever(
        project_id=PROJECT_ID, search_engine_id=data_store_id, max_documents=k
    )

    # The selected Google model uses embedded documents related to the query
    # It parses these documents in order to answer the user question.
    # We use the VertexAI LLM, however other models can be substituted here
    llm = VertexAI(model_name=MODEL, k=k, temperature=temperature)

    # A conversation retrieval chain keeps a history of Q&A / conversation
    # This allows for contextual questions such as "give an example of that (previous response)".
    # The chain is also set to return the source documents used in generating an output
    # This allows for explainability behind model output.
    conversational_retrieval = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=retriever,
        return_source_documents=True
    )
    return conversational_retrieval

In [42]:
import gradio as gr
from langchain.prompts import PromptTemplate

class View:
    def __init__(self, qa_chain=None, vector_store=None):
        if vector_store:
            # condense_question_prompt = PromptTemplate.from_template(TASK_01_PROMPT)
            self.qa_chain = create_qa_chain(
                vector_store,
                # condense_question_prompt
            )
        elif qa_chain:
            self.qa_chain = qa_chain
        else:
            raise Exception("Must pass either qa_chain or vector_store as argument.")

    def q_a(self, question: str, history: list):
        # map history (list of lists) to expected format of chat_history (list of tuples)
        chat_history = map(tuple, history)

        # Query the LLM to get a response
        # First the Q&A chain will collect documents semantically similar to the question
        # Then it will ask the LLM to use this data to answer the user question
        # We also provide chat history as further context
        response = self.qa_chain(
            {
                "question": question,
                "chat_history": chat_history,
            }
        )

        # Format source documents (sources of excerpts passed to the LLM) into links the user can validate
        # Strip index.html so URLs terminate in the parent folder
        # Strip https:// and http:// and replace with https:// to enforce https protocol and catch cases where https:// is present or not present
        sources = [
            "[https://{0}](https://{0})".format(
                doc.metadata["source"].replace("index.html", "").replace("https://", "").replace("http://", "")
            )
            for doc in response["source_documents"]
        ]

        # Return the LLM answer, and list of sources used (formatted as a string)
        return response["answer"], "\n\n".join(sources)

    def submit(self, msg, chatbot):
        # First create a new entry in the conversation log
        msg, chatbot = self.user(msg, chatbot)
        # Then get the chatbot response to the user question
        chatbot = self.bot(chatbot)
        return msg, chatbot

    def user(self, user_message, history):
        # Return "" to clear the user input, and add the user question to the conversation history
        return "", history + [[user_message, None]]

    def bot(self, history):
        # Get the user question from conversation history
        user_message = history[-1][0]
        # Get the response and sources used to answer the user question
        bot_message, bot_sources = self.q_a(user_message, history[:-1])

        # Using a template, format the response and sources together
        bot_template = (
            "{0}\n\n<details><summary><b>Sources</b></summary>\n\n{1}</details>"
        )
        # Place the response into the conversation history and return
        history[-1][1] = bot_template.format(bot_message, bot_sources)
        return history

    def launch_interface(self, share=True, debug=True):
        # Build a simple GradIO app that accepts user input and queries the LLM
        # Then displays the response in a ChatBot interface, with markdown support.
        with gr.Blocks(theme=gr.themes.Base()) as demo:
            # Set a page title
            gr.Markdown("# Custom knowledge worker")
            # Create a chatbot conversation log
            chatbot = gr.Chatbot(label="🤖 knowledge worker")
            # Create a textbox for user questions
            msg = gr.Textbox(
                label="👩‍💻 user input",
                info="Query information from the custom knowledge base.",
            )

            # Align both buttons on the same row
            with gr.Row():
                send = gr.Button(value="Send", variant="primary", size="sm")
                clear = gr.Button(value="Clear History", variant="secondary", size="sm")

            # Submit message on <enter> or clicking "Send" button
            msg.submit(self.submit, [msg, chatbot], [msg, chatbot], queue=False)
            send.click(self.submit, [msg, chatbot], [msg, chatbot], queue=False)

            # Clear chatbot history on clicking "Clear History" button
            clear.click(lambda: None, None, chatbot, queue=False)

        # Create a queue system so multiple users can access the page at once
        demo.queue()
        # Launch the webserver locally
        demo.launch(share=share, debug=debug)


In [43]:
qa_chain_ui = create_qa_vertex_ai_search_chain()
demo = View(qa_chain=qa_chain_ui)
demo.launch_interface()

send: b'GET /startup-events HTTP/1.1\r\nHost: 127.0.0.1:7860\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Mon, 09 Oct 2023 14:33:42 GMT
header: server: uvicorn
header: content-length: 5
header: content-type: application/json
send: b'HEAD / HTTP/1.1\r\nHost: 127.0.0.1:7860\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Mon, 09 Oct 2023 14:33:42 GMT
header: server: uvicorn
header: content-length: 5494
header: content-type: text/html; charset=utf-8
Running on local URL:  http://127.0.0.1:7860
send: b'POST /gradio-initiated-analytics/ HTTP/1.1\r\nHost: api.gradio.app\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 100\r\nContent-Type: application/x-www-form-urlencoded\r\n\r

send: b'POST /gradio-launched-telemetry/ HTTP/1.1\r\nHost: api.gradio.app\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 689\r\nContent-Type: application/x-www-form-urlencoded\r\n\r\n'
send: b'launch_method=inline&is_google_colab=False&is_sharing_on=True&share_url=https%3A%2F%2F4839a30c95b63e15ae.gradio.live&enable_queue=True&show_tips=False&server_name=127.0.0.1&server_port=7860&is_space=False&mode=blocks&version=3.36.1&is_kaggle=False&is_sagemaker=False&using_auth=False&dev_mode=False&show_api=True&show_error=False&title=Gradio&inputs=textbox&inputs=chatbot&inputs=textbox&inputs=chatbot&outputs=textbox&outputs=chatbot&outputs=textbox&outputs=chatbot&outputs=chatbot&targets=textbox&targets=button&targets=button&blocks=markdown&blocks=chatbot&blocks=textbox&blocks=row&blocks=button&blocks=button&blocks=form&events=submit&events=click&events=click&ip_address=139.47.66.38'
reply: 'HTTP/1.1 200 OK\r\n'
he

#### **➡️ Your task:**
 Use the user interface above (which you can also open in a separate tab given the shareable link above) to query your knowledge base.

Try out a few questions from this example Q&A (using the Datatonic web archive):

> 👩‍💻: What is Alphabet?
> 
> 🦜: Alphabet is a holding company that owns Google and other businesses.
> 
> 👩‍💻: Who is the CEO?
> 
> 🦜: The CEO of Alphabet is Sundar Pichai.
> 
> 👩‍💻: What were alphabet revenues in 2021?
>  
> 🦜: Alphabet's revenues in 2021 were $257,637 million.
> 
> 👩‍💻: And in 2022?
>
> 🦜: Alphabet's revenues in 2022 were $282,836 million.

Since we used a `ConversationalRetrievalChain`, we can also correct the model when it gives the wrong response and prompt it to fix it’s mistake, or ask for further detail on a previous response.

**🎉 Congratulations! 🎉** You've created your first chain using LangChain which you can query for general questions in a user interface.
Let's continue extending your knowledge worker in the next task.

# Section 2: Build your own custom knowledge worker end-to-end

We are now going to build a custom knowledge worker end-to-end using the Datatonic website as a knowledge source.

In the previous section, you have learned how to use Vertex AI Search. This is Vertex AI's managed service version of a vector store. As such, Vertex AI manages the document loading, indexing, splitting, embedding and retrieval.

However, in some cases, it is also beneficial to know how to do these steps yourself, using a fully custom solution. Therefore in this next section we are going to learn how to:
* Load documents
* Split documents
* Embed documents
* Ingest them in a vector store
* Use an LLM to answer queries about a/your company knowledge
* Retrieve them based on similarity search

### 2.1 Collecting documents
First, we need to collect our data. To get started fast, we've already downloaded some sample website data upfront. Let's copy the website data from a public Cloud Storage bucket to your local file system

**❗ Note:** Although PaLM supports multiple languages, text embeddings currently work best with English documents.

In [28]:
BUCKET = "dt-gen-ai-hackathon-webarchive"  
LOCAL_FOLDER = "www.datatonic.com.tar.gz"

In [29]:
!gsutil cp gs://{BUCKET}/{LOCAL_FOLDER} . && tar -xzf {LOCAL_FOLDER}

Copying gs://dt-gen-ai-hackathon-webarchive/www.datatonic.com.tar.gz...
 [0 files][  2.3 MiB/  8.4 MiB]                                                |/
Operation completed over 1 objects/8.4 MiB.                                      


If we browse the files we just downloaded, we can see the file structure contains folders and `HTML` files. This is because it is a local replica of the target website, meaning the file paths correlate with real webpages.

#### **➡️ Your task:** 
Run the following cells to *create* the text embeddings based on your downloaded data.

Now, lets embed these files so we can use them in our knowledge worker.

LangChain supports [numerous methods](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html) for loading documents.

We will be using the `DirectoryLoader` and `UnstructuredHTMLLoader` in order to load a pre-compiled archive of your website. This method is similar to the [`RecursiveUrlLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/recursive_url_loader). This document loader searches for subpages of a website and loads each pages content as a document. Additionally, if we only wanted to download a list of URLs without searching for subpages, we could use a [`UnstructuredURLLoader`](https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/url).

**❗ Note:** We're using pre-compiled archives to avoid hitting rate limits during this session, however this content can also be programatically gathered routinely to collect new blog posts / press releases.

#### **➡️ Your task:**
 Read the linked resources in the `Introduction to LangChain` step and study the following code cells as they provide reusable LangChain code for your knowledge worker.

In the code cell below, we parse the directory to find `HTML` files and load their contents using `UnstructuredHTMLLoader`.

In [30]:
from langchain.document_loaders import DirectoryLoader, UnstructuredHTMLLoader


def load_documents(source_dir):
    # Load the documentation using a HTML parser
    loader = DirectoryLoader(
        source_dir,
        glob="**/*.html",
        loader_cls=UnstructuredHTMLLoader,
        show_progress=True,
    )
    documents = loader.load()

    print(f"Loaded: {len(documents)} documents from {source_dir}.")

    return documents

### 2.2 Creating or loading embeddings

Creating embeddings each time we use our app is time-consuming and expensive.
By persisting the vector store database after embedding, we can load the saved embeddings for use in another session.

#### **➡️ Your task:** 
Study and execute the following code cells. Note that after the documents have been loaded, they are split into shards using the `RecursiveCharacterTextSplitter` function. 

In [31]:
# @title Set the name of your vectorstore { run: "auto", display-mode: "form" }
# @markdown This variable can be left as default for this task.
PERSIST_DIR = "chromadb"  # @param {type:"string"}

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import VertexAIEmbeddings


def create_embeddings(source_dir):
    documents = load_documents(source_dir=source_dir)

    # We use Google embeddings model, however other models can be substituted here
    embeddings = VertexAIEmbeddings(model_name='textembedding-gecko@001')

    # Individual documents will often exceed the token limit.
    # By splitting documents into chunks of 1000 token
    # These chunks fit into the token limit alongside the user prompt
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    vector_store = Chroma.from_documents(
        documents=texts, embedding=embeddings, persist_directory=PERSIST_DIR
    )

    # Persist the ChromaDB locally, so we can reload the script without expensively re-embedding the database
    vector_store.persist()

In [34]:
create_embeddings(
    source_dir=LOCAL_FOLDER.rstrip('.tar.gz')
)  # creates the vector DB and saves it locally. Be patient, this may take a few minutes.
print(f"Created new vectorstore in dir {PERSIST_DIR}.")

  0%|          | 0/160 [00:00<?, ?it/s]INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
  1%|          | 1/160 [00:02<07:05,  2.68s/it]INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
  2%|▎         | 4/160 [00:02<01:23,  1.86it/s]INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
  4%|▍         | 7/160 [00:03<00:44,  3.44it/s]INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstr

Loaded: 160 documents from www.datatonic.com.


INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
INFO:clickhouse_connect.driver.ctypes:Successfully imported ClickHouse Connect C data optimizations
INFO:clickhouse_connect.json_impl:Using orjson library for writing JSON byte strings
INFO:chromadb.db.duckdb:No existing DB found in chromadb, skipping load
INFO:chromadb.db.duckdb:No existing DB found in chromadb, skipping load


send: b'POST /batch/ HTTP/1.1\r\nHost: app.posthog.com\r\nUser-Agent: posthog-python/3.0.1\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Type: application/json\r\nContent-Length: 563\r\n\r\n'
send: b'{"batch": [{"properties": {"chroma_context": {"chroma_version": "0.3.26", "server_context": "None", "chroma_db_impl": "chromadb.db.duckdb.DuckDB", "chroma_api_impl": "chromadb.api.local.LocalAPI", "chroma_server_ssl_enabled": false}, "$lib": "posthog-python", "$lib_version": "3.0.1", "$geoip_disable": true}, "timestamp": "2023-10-09T14:25:39.234038+00:00", "context": {}, "distinct_id": "0826c9e4-048d-4eac-8005-c69f998c198d", "event": "client_start"}], "sentAt": "2023-10-09T14:25:39.735266+00:00", "api_key": "phc_YeUxaojbKk5KPi8hNlx1bBKHzuZ4FDtl67kH1blv8Bh"}'
reply: 'HTTP/1.1 200 OK\r\n'
header: Content-Type: application/json
header: Transfer-Encoding: chunked
header: Connection: keep-alive
header: Date: Mon, 09 Oct 2023 14:25:39 GMT
header: Server: g

INFO:chromadb.db.duckdb:Persisting DB to disk, putting it in the save folder: chromadb


Created new vectorstore in dir chromadb.


#### **➡️ Your task:** 
Run the following cells to *load* the text embeddings.

In [35]:
def load_embeddings(persist_directory):
    # We use VertexAI embeddings model, however other models can be substituted here
    embeddings = VertexAIEmbeddings(model_name='textembedding-gecko@001')

    # Creating embeddings with each re-run is highly inefficient and costly.
    # We instead aim to embed once, then load these embeddings from storage.
    vector_store = Chroma(
        embedding_function=embeddings,
        persist_directory=persist_directory,
    )

    return vector_store

In [36]:
vector_store = load_embeddings(
    PERSIST_DIR
)  # loads the vector DB from the local file system.
print(f"Loaded {PERSIST_DIR} as vectorstore.")

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.db.duckdb:loaded in 1213 embeddings
INFO:chromadb.db.duckdb:loaded in 1 collections
INFO:chromadb.db.duckdb:collection with name langchain already exists, returning existing collection


Loaded chromadb as vectorstore.


**🎉 Congratulations! 🎉** You've created text embeddings from your company data and stored them successfully in a local vector database.
Now, you'll shift your focus to implementing the actual LLM by creating a chain using LangChain.

## 2.3 Creating the Conversational Q&A Chain

In this section, you'll create a chain which will be able to provide an answer given a question from a user.
To understand the purpose of chains, you can read about chains in the [LangChain documentation](https://docs.langchain.com/docs/).


In [39]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import VertexAI


def create_qa_chain(vector_store, k=4, temperature=0.0):
    """ Create a Q&A conversation chain using the VertexAI LLM.

    Arguments:
        vector_store (object): The vectorstore containing our knowledge.
        condense_question_prompt (PromptTemplate): The prompt template used to prompt engineer our LLM to respond in a certain tone, etc.
        k (int): the 'k' value indicates the number of sources to use per query. 'k' as in 'k-nearest-neighbours' to the query in the embedding space.
        temperature (float): the degree of randomness introduced into the LLM response.
    """

    # A vector store retriever relates queries to embedded documents
    retriever = vector_store.as_retriever(k=k)

    # The selected Google model uses embedded documents related to the query
    # It parses these documents in order to answer the user question.
    # We use the VertexAI LLM, however other models can be substituted here
    model = VertexAI(model_name='text-bison@001', temperature=temperature)

    # A conversation retrieval chain keeps a history of Q&A / conversation
    # This allows for contextual questions such as "give an example of that (previous response)".
    # The chain is also set to return the source documents used in generating an output
    # This allows for explainability behind model output.
    chain = ConversationalRetrievalChain.from_llm(
        llm=model,
        retriever=retriever,
        return_source_documents=True
    )

    return chain

## 2.4 Building the user interface

As building a UI is outside of the scope for this hackathon, a templated GUI using [Gradio](https://gradio.app/) is provided for your knowledge worker. This is the same UI as in the previous section.

#### **➡️ Your task:** Execute the cells below to launch the user interface.

**❗ Note:** The following cell will run until manually stopped. Remember to halt it before moving onto the next task.

In [40]:
qa_chain = create_qa_chain(
    vector_store=vector_store
)

demo = View(qa_chain=qa_chain)
demo.launch_interface()

send: b'GET /startup-events HTTP/1.1\r\nHost: 127.0.0.1:7860\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Mon, 09 Oct 2023 14:31:48 GMT
header: server: uvicorn
header: content-length: 5
header: content-type: application/json
send: b'HEAD / HTTP/1.1\r\nHost: 127.0.0.1:7860\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Mon, 09 Oct 2023 14:31:48 GMT
header: server: uvicorn
header: content-length: 5491
header: content-type: text/html; charset=utf-8
Running on local URL:  http://127.0.0.1:7860
send: b'POST /gradio-initiated-analytics/ HTTP/1.1\r\nHost: api.gradio.app\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 100\r\nContent-Type: application/x-www-form-urlencoded\r\n\r

send: b'POST /gradio-launched-telemetry/ HTTP/1.1\r\nHost: api.gradio.app\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 689\r\nContent-Type: application/x-www-form-urlencoded\r\n\r\n'
send: b'launch_method=inline&is_google_colab=False&is_sharing_on=True&share_url=https%3A%2F%2F9a0b8840e3dfb79871.gradio.live&enable_queue=True&show_tips=False&server_name=127.0.0.1&server_port=7860&is_space=False&mode=blocks&version=3.36.1&is_kaggle=False&is_sagemaker=False&using_auth=False&dev_mode=False&show_api=True&show_error=False&title=Gradio&inputs=textbox&inputs=chatbot&inputs=textbox&inputs=chatbot&outputs=textbox&outputs=chatbot&outputs=textbox&outputs=chatbot&outputs=chatbot&targets=textbox&targets=button&targets=button&blocks=markdown&blocks=chatbot&blocks=textbox&blocks=row&blocks=button&blocks=button&blocks=form&events=submit&events=click&events=click&ip_address=139.47.66.38'
reply: 'HTTP/1.1 200 OK\r\n'
he

#### **➡️ Your task:**
 Use the user interface above (which you can also open in a separate tab given the shareable link above) to query your knowledge base.

Try out a few questions from this example Q&A (using the Datatonic web archive):

> 👩‍💻: What is Datatonic?
> 
> 🦜: Datatonic is a data consultancy enabling companies to make better business decisions with the power of Modern Data Stack and MLOps.
> 
> 👩‍💻: Summarise the web article on Greentonic.
> 
> 🦜: Greentonic is Datatonic's sustainability initiative.
> 
> 👩‍💻: How is Datatonic being sustainable?
> 
> 🦜: Datatonic is committed to sustainability and has a number of initiatives in place to reduce its environmental impact. These include:
>    * Using renewable energy sources
>    * Reducing our carbon footprint
>    * Promoting sustainable practices in our supply chain
>    * Supporting environmental charities
>
> We believe that sustainability is essential for the future of our planet and we are committed to doing our part to make a difference.

Again, since we used a `ConversationalRetrievalChain`, we can also correct the model when it gives the wrong response and prompt it to fix it’s mistake, or ask for further detail on a previous response.

## 2.5: Extending the knowledge base

As mentioned, it is possible to extend the knowledge base with additional documents.
This is useful for updating a knowledge base with new information without having to re-embed established knowledge from scratch.

If you wanted to build a knowledge worker with another document type, for instance [Microsoft Word](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/microsoft_word.html) documents, you would update the `load_documents()` function according to the documentation for that document type loader.

**❗ Note:** if you're looking to deploy a knowledge worker with several knowledge bases, an [Embedding Router Chain](https://python.langchain.com/docs/modules/chains/foundational/router#embeddingrouterchain), which combines several knowledge workers with discrete knowledge bases into a single chain which selects the best worker for the query.

#### **➡️ Your task:**
Extend the knowledge worker with new PDFs documents.
1. Download some PDFs locally.
2. Load the new documents.
3. Add new documents to the existing vector store using the `.add_documents(documents=...)` method.


In [74]:
# Downloading documents locally
!gsutil -m cp -r "gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021" .

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021/20040630_google_10Q.pdf...
Copying gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021/20040930_google_10Q.pdf...
Copying gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021/2004Q3_earnings.pdf...
Copying gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021/2004Q4_earnings_google.pdf...
Copying gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021/2004_google_annual_report.pdf...
Copying gs://dt-gen-ai-hackathon-pdf-datasets/alphabet_investor_pdfs_2004_2021/20050331_google_10Q.pdf...
Copying 

Use LangChains's support for `PyPDFLoader` to load a few new PDFs into the same vector store. We suggest using these two:
* `alphabet-investor-pdfs/2004_google_annual_report.pdf`
* `alphabet-investor-pdfs/2010Q1_earnings_google.pdf`

In [ ]:
# Your code here

In [75]:
# ANSWER
from langchain.document_loaders import PyPDFLoader

def load_pdf_documents(filepath):
    loader = PyPDFLoader(filepath)
    pages = loader.load_and_split()

    # add this document(s) to the vector store
    vector_store.add_documents(pages)

    # "save" the new vector store back to the file system
    vector_store.persist()

    print("Finished adding new documents to the vectorstore.")

In [76]:
load_pdf_documents(
    filepath="alphabet_investor_pdfs_2004_2021/2004_google_annual_report.pdf"
)

INFO:chromadb.db.duckdb:Persisting DB to disk, putting it in the save folder: chromadb


send: b'POST /batch/ HTTP/1.1\r\nHost: app.posthog.com\r\nUser-Agent: posthog-python/3.0.1\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Type: application/json\r\nContent-Length: 643\r\n\r\n'
send: b'{"batch": [{"properties": {"collection_uuid": "b0694d12-0e1b-420d-80cc-5261d22ce82b", "add_amount": 166, "chroma_context": {"chroma_version": "0.3.26", "server_context": "None", "chroma_db_impl": "chromadb.db.duckdb.DuckDB", "chroma_api_impl": "chromadb.api.local.LocalAPI", "chroma_server_ssl_enabled": false}, "$lib": "posthog-python", "$lib_version": "3.0.1", "$geoip_disable": true}, "timestamp": "2023-10-09T15:38:28.801417+00:00", "context": {}, "distinct_id": "0826c9e4-048d-4eac-8005-c69f998c198d", "event": "collection_add"}], "sentAt": "2023-10-09T15:38:29.148306+00:00", "api_key": "phc_YeUxaojbKk5KPi8hNlx1bBKHzuZ4FDtl67kH1blv8Bh"}'
reply: 'HTTP/1.1 200 OK\r\n'
header: Content-Type: application/json
header: Transfer-Encoding: chunked
header: Conn

#### **➡️ Your task:**
Rerun the app and try asking questions using knowledge from your newly added documents.

**❗ Note:** The following cell will run until manually stopped. Remember to halt it before moving onto the next task.

Try asking questions about different topics coming from different sources. With this you will see the value of a custom vector store where you can ingest multiple data types and sources. For example, try asking:
* About Datatonic's sustainability initiative
* About Alphabet's revenue performance in 2004 and 2010
* Then about some of Datatonic's case studies again

In [77]:
qa_chain = create_qa_chain(
    vector_store=vector_store
)

demo = View(qa_chain=qa_chain)
demo.launch_interface()

send: b'GET /startup-events HTTP/1.1\r\nHost: 127.0.0.1:7860\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Mon, 09 Oct 2023 15:38:37 GMT
header: server: uvicorn
header: content-length: 5
header: content-type: application/json
send: b'HEAD / HTTP/1.1\r\nHost: 127.0.0.1:7860\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Mon, 09 Oct 2023 15:38:37 GMT
header: server: uvicorn
header: content-length: 5494
header: content-type: text/html; charset=utf-8
Running on local URL:  http://127.0.0.1:7860
send: b'POST /gradio-initiated-analytics/ HTTP/1.1\r\nHost: api.gradio.app\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 100\r\nContent-Type: application/x-www-form-urlencoded\r\n\r

send: b'POST /gradio-launched-telemetry/ HTTP/1.1\r\nHost: api.gradio.app\r\nUser-Agent: python-requests/2.31.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 689\r\nContent-Type: application/x-www-form-urlencoded\r\n\r\n'
send: b'launch_method=inline&is_google_colab=False&is_sharing_on=True&share_url=https%3A%2F%2F133857b4ba4d7483b4.gradio.live&enable_queue=True&show_tips=False&server_name=127.0.0.1&server_port=7860&is_space=False&mode=blocks&version=3.36.1&is_kaggle=False&is_sagemaker=False&using_auth=False&dev_mode=False&show_api=True&show_error=False&title=Gradio&inputs=textbox&inputs=chatbot&inputs=textbox&inputs=chatbot&outputs=textbox&outputs=chatbot&outputs=textbox&outputs=chatbot&outputs=chatbot&targets=textbox&targets=button&targets=button&blocks=markdown&blocks=chatbot&blocks=textbox&blocks=row&blocks=button&blocks=button&blocks=form&events=submit&events=click&events=click&ip_address=139.47.66.38'
reply: 'HTTP/1.1 200 OK\r\n'
he

**🎉 Congratulations! 🎉** You've extended your knowledge to creating text embedding from a variety of sources - whether it's public data from your company's website or unstructured documents!

## 2.6 Generating text over a vector index

We can utilise our embedded documents for more than just Q&A.
In tasks 1 and 2, we used the embedded documents as context for answering user queries, but in this task we will use it to generate original content using this knowledge base as a source of information and style.

The concept of this use case is to generate ideas for new blogs, utilising knowledge and style information contained in the existing company website data.
We can use Generative AI for creative ideation, too!
Let's demonstrates the possibilities for human-computer interaction (HCI) apps in this task.

In [ ]:
# @title Set LLM temperature { run: "auto", display-mode: "form" }
# @markdown Temperature controls the degree of creativity / randomness introduced into the LLM.
temperature = 0.7  # @param {type:"slider", min:0, max:1, step:0.01}

In [ ]:
SECTION_02_TEMPLATE = """\
Using the provided context, write the outline of a company blog post.
Include a bullet-point list of the main talking points, and a brief summary of the overall blog.

Context: {context}
Topic: {topic}
"""

SECTION_02_PROMPT = PromptTemplate.from_template(SECTION_02_TEMPLATE)

In [ ]:
from langchain.chains import LLMChain

model = VertexAI(model_name='text-bison@001', temperature=temperature)

chain = LLMChain(llm=model, prompt=SECTION_02_PROMPT)

In [ ]:
def generate_blog_outline(topic: str, k: int):
    # search for 'k' nearest documents related to our topic.
    docs = vector_store.similarity_search(topic, k=k)

    # associate topic with the content of each document to generate inputs
    inputs = [{"context": doc.page_content, "topic": topic} for doc in docs]

    # generate blog outline
    output = chain.apply(inputs)

    return output

#### **➡️ Your task:** 
Create ideas for a new blog post.
Try adjusting the title of the post to generate new ideas!

In [ ]:
# @title Set blog prompt / title { run: "auto", display-mode: "form" }
# @markdown Be descriptive, as the LLM will collect semantically similar sources as inspiration.
BLOG_TITLE = "How we're making our business more sustainable"  # @param {type:"string"}

In [ ]:
from IPython.display import display, Markdown

# generate variations of blog posts on the topic provided, based on the 4 most relevant documents
output = generate_blog_outline(BLOG_TITLE, k=4)
markdown = ""

for i, blog in enumerate(output):
    markdown += f"# #{i} {BLOG_TITLE}\n{blog['text']}\n\n"

display(Markdown(markdown))

#### **➡️ Your task:**
Update the `SECTION_02_PROMPT`, `temperature` and `BLOG_TITLE` variables to create new types of content.

**🎉 Congratulations! 🎉** You've completed task 3 and generated ideas for future blog posts!
Continue with the next section to explore more possibilities and ideas using LangChain.

# Bonus Track: Extending the chain

So far you've created two types of chains:

### LLMChain

The `LLMChain` is a simple chain that adds some functionality around language models.
It is used widely throughout LangChain, including in other chains and agents.

An LLMChain consists of a **PromptTemplate** and a **language model** (either an LLM or chat model).
It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.

```python
chain = LLMChain(llm=model, prompt=PROMPT)
```

### ConversationalRetrievalChain

The `ConversationalRetrievalQA` chain builds on RetrievalQAChain to provide a chat history component.

It first combines the chat history (either explicitly passed in or retrieved from the provided memory) and the question into a standalone question, then looks up relevant documents from the retriever, and finally passes those documents and the question to a question answering chain to return a response.

To create one, you will need a retriever.
In the below example, we will create one from a vector store, which can be created from embeddings.

```python
retriever = vector_store.as_retriever(k=k)
model = VertexAI(temperature=temperature)
chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    return_source_documents=True,
    condense_question_prompt=TASK_01_PROMPT,
)
```

### Explore more chains

**➡️ Your task:** Firm up your knowledge about the two chains used in this notebook [here](https://python.langchain.com/docs/modules/chains/foundational/llm_chain) and [here](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db).
In which scenarios should you apply either of them?
What are their limitations?

*The LLMChain is useful when ...*

*It's limitations are ...*

*The ConversationalRetrievalChain is useful when ...*

*It's limitations are ...*

**➡️ Your task:** Read about more types of chains in the [official LangChain documentation](https://python.langchain.com/docs/modules/chains/additional/).
We recommend the **Sequential chain** and **Self-critique chain with constitutional AI**.
How can you extend your conversational knowledge worker which is currently based on the `ConversationalRetrievalChain`?
Summarise your idea either using pseudo code or actual code if you've time!
Overall we would like to you to consider:

**Idea + idea description:**

- *The idea is ...*
- *What it is ...*

**Problem it solves + impact:**

- *It would solve the following challenge ...*
- *The volume or value of the impact would be ...*

**Approach + Next steps:**

- *Next steps would be ...*

**❗ Note:** Do you have any other ideas (even outside of implementing a knowledge worker)?
Feel free to ideate about another use case which is relevant to your industry or company!

In [ ]:
# Write your ideas/code here

Try asking the same questions you used to query your knowledge worker. Does it answer the question in the same way? Does it give more or less detail? What are the immediate differences between solutions?

**🎉 Congratulations! 🎉** You've gotten started with Enterprise Search. 

# Conclusion

## What have we built?

In this session, we have built a knowledge worker use-case for accessing your complex information using Generative AI.
This concept can be extended into a fully-fledged tool that can unlock the value of your data for customers or internal use.

## Going further...

This workshop has introduced all the LangChain knowledge required to create a knowledge worker.
The next steps for moving this project from development to production are discussed below.

## Decoupling LangChain from Gradio

It is not necessary to run LangChain within a GradI/O app.
Decoupling LangChain into a separate API has several benefits:
1. We can deploy scalable servers / Docker containers
2. Simplified code - a frontend-backend loose coupling can lead to simpler code, which is ease to update and maintain.
3. If a more professional user interface is needed, such as a native React app.
Replacing GradI/O is a straightforward process - FastAPI can be called from javascript, etc., allowing you to move beyond Python frontend frameworks.

An example of this separation can be found on the GitHub repository, using FastAPI to create a simple LangChain API server and Poetry to manage separate server environments.

## Deploying on Google Cloud

Once we have decoupled our frontend and backend code, we can deploy the project onto Google Cloud.

This reference architecture diagrams mirror the flow diagrams we first introducted in the workshop introduction. Using Google Cloud, we can create production pipelines for creating / updating vector databases, and deploy a knowledge worker API (which can be connected to a web UI, Slack bot, etc.).

**Example architecture: Ingestion**
![A typical ingestion chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/7f37d477b18ace5912d34b0574512559d7a457ed/assets/knowledge-worker-gcp-ingestion-pipeline.png?raw=true)

By creating a pipeline for data ingestion, we can continue to extend the knowledge base of our knowledge worker as you produce new documents and documentation.

**Example architecture: Inference**
![A typical ingestion chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/7f37d477b18ace5912d34b0574512559d7a457ed/assets/knowledge-worker-gcp-inference-pipeline.png?raw=true)

By creating a pipeline for inference, we can leverage the power of Google Cloud to provide a highly reliable and scalable API that can power a variety of applications.

**🎉 Congratulations! 🎉** You've completed this notebook!
Now it's time to embark your Generative AI journey and ideate about use cases which can benefit your company in conjunction or in addition to your first knowledge worker.